<a href="https://colab.research.google.com/github/mgnarag/binarization-autoencoder/blob/main/Traditional_binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/one quadrant/"

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import matplotlib.pyplot as plt

import cv2

In [25]:
def metrics(gt,thresh1):

  psnr = cv2.PSNR(gt, thresh1)

  # Ensure images are binary
  image1 = gt
  image2 = thresh1
  _, image1 = cv2.threshold(image1, 127, 1, cv2.THRESH_BINARY)
  _, image2 = cv2.threshold(image2, 127, 1, cv2.THRESH_BINARY)
  image1 = 1 - image1
  image2 = 1 - image2

  #image1 = image1[0:7168, 0:9728]
  #image2 = image2[0:7168, 0:9728]

  # Calculate True Positive, False Positive, False Negative
  TP = np.sum(np.logical_and(image1, image2))
  FP = np.sum(np.logical_and(image1 == 0, image2 == 1))
  FN = np.sum(np.logical_and(image1 == 1, image2 == 0))

  # Calculate precision, recall, and F1 score
  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  f1_score = 2 * (precision * recall) / (precision + recall)

  intersection = np.sum(np.logical_and(image1, image2))
  union = np.sum(np.logical_or(image1, image2))
  iou = intersection / union

  return f1_score, iou, psnr


def simple_thresh(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  for i in range(0,255):
    ret,thresh1 = cv2.threshold(test,i,255,cv2.THRESH_BINARY)
    M = metrics(gt,thresh1)
    F1.append(M[0])
    IOU.append(M[1])
    PSNR.append(M[2])
  index = np.nanargmax(F1)

  #saving:
  ret,thresh1 = cv2.threshold(test,index,255,cv2.THRESH_BINARY)
  final = Image.fromarray(thresh1)
  final.save(file_path + "/6 Traditional Thresh Results/"+"[1] SIMPLE_pix_" + str(index)+".png")

  return F1[index], IOU[index], PSNR[index], index

def otsu_thresh(gt,test):
  ret,thresh1 = cv2.threshold(test,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  M = metrics(gt,thresh1)
  final = Image.fromarray(thresh1)
  final.save(file_path +  "/6 Traditional Thresh Results/"+"[2] OTSU.png")
  return M[0], M[1], M[2]

In [4]:
def adaptive_thresh_gaussian(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  kernel = [i for i in range(3, 128, 4)]
  for k in kernel:
    F1_per_kernel = []
    IOU_per_kernel = []
    PSNR_per_kernel = []
    print(k)
    for i in range(0,100):
      #print(i)
      thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,k,i)
      M = metrics(gt,thresh1)
      F1_per_kernel.append(M[0])
      IOU_per_kernel.append(M[1])
      PSNR_per_kernel.append(M[2])

    F1.append(F1_per_kernel)
    IOU.append(IOU_per_kernel)
    PSNR.append(PSNR_per_kernel)

  F1 = np.array(F1)
  flat_index = np.nanargmax(F1)
  index = np.unravel_index(flat_index, F1.shape)

  #saving:
  thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,kernel[index[0]], index[1])
  final = Image.fromarray(thresh1)
  final.save(file_path + "/6 Traditional Thresh Results/"+ "[3] ADAPTIVE_GAUSSIAN_kernel_"+str(kernel[index[0]])+ "pix_"+str(index[1])+".png")

  return F1[index[0]][index[1]], IOU[index[0]][index[1]], PSNR[index[0]][index[1]], kernel[index[0]], index[1]

def adaptive_thresh_mean(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  kernel = [i for i in range(3, 128, 4)]
  for k in kernel:
    F1_per_kernel = []
    IOU_per_kernel = []
    PSNR_per_kernel = []
    print(k)
    for i in range(0,100):
      thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,k,i)
      #print(i)
      M = metrics(gt,thresh1)
      F1_per_kernel.append(M[0])
      IOU_per_kernel.append(M[1])
      PSNR_per_kernel.append(M[2])

    F1.append(F1_per_kernel)
    IOU.append(IOU_per_kernel)
    PSNR.append(PSNR_per_kernel)

  F1 = np.array(F1)
  flat_index = np.nanargmax(F1)
  index = np.unravel_index(flat_index, F1.shape)

  #saving:
  thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,kernel[index[0]], index[1])
  final = Image.fromarray(thresh1)
  final.save(file_path +  "/6 Traditional Thresh Results/"+"[4] ADAPTIVE_MEAN_kernel_"+str(kernel[index[0]])+ "pix_"+str(index[1])+".png")

  return F1[index[0]][index[1]], IOU[index[0]][index[1]], PSNR[index[0]][index[1]], kernel[index[0]], index[1]

In [26]:
gt = cv2.imread(file_path+ '4 Testing/ground_truth.png', cv2.IMREAD_GRAYSCALE)
test = cv2.imread(file_path+ '4 Testing/1.png', cv2.IMREAD_GRAYSCALE)
#for simple thresh:
otsu = otsu_thresh(gt,test)
print('simple otsu = ', otsu)
simple = simple_thresh(gt,test)
print('simple thresh = ', simple)

simple otsu =  (0.3320642955103536, 0.20839995626427807, 5.838131434810624)


<ipython-input-25-cd068d63beef>:39: RuntimeWarning: invalid value encountered in scalar divide
  precision = TP / (TP + FP)


simple thresh =  (0.9515394272976996, 0.9376721800412845, 21.936912206820548, 48)


In [6]:
#for adaptive thresh:
gaussian = adaptive_thresh_gaussian(gt,test)
print('adaptive thresh = ', gaussian)
mean = adaptive_thresh_mean(gt,test)
print('adaptive thresh = ', mean)

3
7
11
15
19
23
27
31
35
39
43
47
51
55
59
63
67
71
75
79
83
87
91
95
99
103
107
111
115
119
123
127
adaptive thresh =  (0.7063200450157411, 0.768041421683472, 18.462075415751606, 127, 18)
3
7
11
15
19
23
27
31
35
39
43
47
51
55
59
63
67
71
75
79
83
87
91
95
99
103
107
111
115
119
123
127
adaptive thresh =  (0.706730208137009, 0.7917091115210428, 18.94980608236171, 127, 20)


In [7]:
simple_ = np.array(simple)
otsu_ = np.array(otsu)
gaussian_ = np.array(gaussian)
mean_ = np.array(mean)


# Creating DataFrames from arrays
df1 = pd.DataFrame({'Simple': simple_})
df2 = pd.DataFrame({'Otsu': otsu_})
df3 = pd.DataFrame({'Gaussian': gaussian_})
df4 = pd.DataFrame({'Mean': mean_})

# Concatenating DataFrames along columns (axis=1)
metric = pd.concat([df1, df2, df3, df4], axis=1)
metric = metric.T

headers = ['F1 score','IOU', 'PSNR', 'pixel/kerner', 'c parameter' ]
metric.columns = headers

metric.to_csv(file_path +  "/6 Traditional Thresh Results/"+ 'Metric_traditional.csv')


print(metric)

          F1 score       IOU       PSNR  pixel/kerner  c parameter
Simple    0.709170  0.937672  21.936912          48.0          NaN
Otsu      0.570384  0.208400   5.838131           NaN          NaN
Gaussian  0.706320  0.768041  18.462075         127.0         18.0
Mean      0.706730  0.791709  18.949806         127.0         20.0


In [27]:
from sklearn.metrics import f1_score
import numpy as np
image1 = cv2.imread(file_path+ '4 Testing/ground_truth.png')
#image2 = cv2.imread(file_path+ '5 Tested/output v8.png')
#image2 = cv2.imread(file_path+ "/6 Traditional Thresh Results/"+"[3] ADAPTIVE_GAUSSIAN_kernel_127pix_18.png")
image2 = cv2.imread(file_path+ "/6 Traditional Thresh Results/"+"[1] SIMPLE_pix_48.png")
# Assuming you have two binary images represented as numpy arrays
# Let's say image1 and image2

# Ensure images are binary
_, image1 = cv2.threshold(image1, 127, 1, cv2.THRESH_BINARY)
_, image2 = cv2.threshold(image2, 127, 1, cv2.THRESH_BINARY)
image1 = 1 - image1
image2 = 1 - image2

#image1 = image1[0:7168, 0:9728]
#image2 = image2[0:7168, 0:9728]

# Calculate True Positive, False Positive, False Negative
TP = np.sum(np.logical_and(image1, image2))
FP = np.sum(np.logical_and(image1 == 0, image2 == 1))
FN = np.sum(np.logical_and(image1 == 1, image2 == 0))

# Calculate precision, recall, and F1 score
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


intersection = np.sum(np.logical_and(image1, image2))
union = np.sum(np.logical_or(image1, image2))
iou = intersection / union
print("IOU", iou)

Precision: 0.9193546922517826
Recall: 0.9860593567707099
F1 Score: 0.9515394272976996
IOU 0.9075586169589607
